# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [2]:
wine_dataset = load_wine()
X = wine_dataset.data[:, [9, 11, 12]]
y = wine_dataset.target


x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=17)

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [3]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [4]:
skf = StratifiedKFold(n_splits=5, random_state=17, shuffle=True).get_n_splits(X, y)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
lreg = LogisticRegression(random_state=17, multi_class='ovr')
lreg_model = lreg.fit(x_train, y_train)
lreg_prediction = lreg_model.predict(x_test)
lreg_accuracy = accuracy_score(y_test, lreg_prediction)

mean_lreg_cv_score = cross_val_score(lreg_model, X, y, cv=skf).mean()


print(f'Доля правильных ответов на тестовой выборке: \t \t{lreg_accuracy}\n\
Средняя доля правильных ответов на кросс-валидации: \t{mean_lreg_cv_score}')


Доля правильных ответов на тестовой выборке: 	 	0.9111111111111111
Средняя доля правильных ответов на кросс-валидации: 	0.9442857142857143


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [7]:
from sklearn.svm import SVC

In [8]:
svc = SVC(random_state=17)
svc_model = svc.fit(x_train, y_train)
svc_prediction = svc_model.predict(x_test)
svc_accuracy = accuracy_score(y_test, svc_prediction)


print(f'Доля правильных ответов на тестовой выборке: \t{svc_accuracy}')





Доля правильных ответов на тестовой выборке: 	0.6222222222222222


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [9]:
from sklearn.metrics import classification_report, confusion_matrix

In [17]:
print('Матрица ошибок логистической регрессии: ')
print(confusion_matrix(y_test, lreg_prediction))
print('-----------------------------------------')
print('Матрица ошибок SVC модели: ')
print(confusion_matrix(y_test, svc_prediction))



Матрица ошибок логистической регрессии: 
[[ 9  0  0]
 [ 0 19  0]
 [ 0  4 13]]
-----------------------------------------
Матрица ошибок SVC модели: 
[[ 9  0  0]
 [ 0 19  0]
 [ 2 15  0]]


* Модель логистической регрессии сильнее всего путает 2 и 1 классы;
* Модель SVC сильнее всего путает 2 и 1 классы.

### 5.
Для каждой модели выведите classification report.

In [19]:
print('Логистическая регрессия: ')
print(classification_report(y_test, lreg_prediction))
print('-----------------------------------------------------')
print('SVC: ')
print(classification_report(y_test, svc_prediction))





Логистическая регрессия: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      1.00      0.90        19
           2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45

-----------------------------------------------------
SVC: 
              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       0.56      1.00      0.72        19
           2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45



C:\Users\larin\anaconda3\envs\MyEnv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\larin\anaconda3\envs\MyEnv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\larin\anaconda3\envs\MyEnv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
